### Predicting number of dengi cases 
 - changed the number of iterations

In [2]:
from pyspark import keyword_only
from pyspark.ml import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import *
from pyspark.ml.classification import *
from pyspark.ml.clustering import *
from pyspark.ml.evaluation import *
from pyspark.ml.tuning import *
from pyspark.ml.param.shared import *
from pyspark.ml.param import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from HTMLParser import HTMLParser
from math import sqrt
from math import isnan
from datetime import datetime
import numpy
import re
import random

In [3]:
dengue_features_train_schema = StructType([
  StructField('city', StringType(), True),
  StructField('year', FloatType(), True),
  StructField('weekofyear', FloatType(), True),
  StructField('week_start_date', DateType(), True),
  StructField('ndvi_ne', FloatType(), True),
  StructField('ndvi_nw', FloatType(), True),
  StructField('ndvi_se', FloatType(), True),
  StructField('ndvi_sw', FloatType(), True),
  StructField('precipitation_amt_mm', FloatType(), True),
  StructField('reanalysis_air_temp_k', FloatType(), True),
  StructField('reanalysis_avg_temp_k', FloatType(), True),
  StructField('reanalysis_dew_point_temp_k', FloatType(), True),
  StructField('reanalysis_max_air_temp_k', FloatType(), True),
  StructField('reanalysis_min_air_temp_k', FloatType(), True),
  StructField('reanalysis_precip_amt_kg_per_m2', FloatType(), True),
  StructField('reanalysis_relative_humidity_percent', FloatType(), True),
  StructField('reanalysis_sat_precip_amt_mm', FloatType(), True),
  StructField('reanalysis_specific_humidity_g_per_kg', FloatType(), True),
  StructField('reanalysis_tdtr_k', FloatType(), True),
  StructField('station_avg_temp_c', FloatType(), True),
  StructField('station_diur_temp_rng_c', FloatType(), True),
  StructField('station_max_temp_c', FloatType(), True),
  StructField('station_min_temp_c', FloatType(), True),
  StructField('station_precip_mm', FloatType(), True)
])

dengue_labels_train_schema = StructType([
  StructField('city', StringType(), True),
  StructField('year', FloatType(), True),
  StructField('weekofyear', FloatType(), True),
  StructField('total_cases', FloatType(), True)
])



In [4]:
train = spark.read.schema(dengue_features_train_schema).csv("s3a://data/dengai/dengue_features_train.csv" , header = True)
label_dataset = spark.read.schema(dengue_labels_train_schema).csv('s3a://data/dengai/dengue_labels_train.csv', header = True)

In [5]:
train_labels = train.join(label_dataset , ['city' , 'year' , 'weekofyear'])

In [6]:
train_labels.select('city').distinct().show()

+----+
city|
+----+
 iq|
 sj|
+----+

In [7]:
def numerical_cities(city):
  if city == 'iq':
    return '1'
  else:
    return '2'
  
transform_city = udf ( numerical_cities , StringType())

In [8]:
train_labels_transform_city = train_labels.withColumn('num_city', transform_city('city'))
train_labels_transform_city = train_labels_transform_city.drop('city')
display(train_labels_transform_city)

year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,num_city
1990.0,18.0,1990-04-30,0.1226,0.103725,0.1984833,0.1776167,12.42,297.57285,297.74286,292.41428,299.8,295.9,32.0,73.365715,12.42,14.012857,2.6285715,25.442858,6.9,29.4,20.0,16.0,4.0,2
1990.0,19.0,1990-05-07,0.1699,0.142175,0.1623571,0.1554857,22.82,298.21143,298.44287,293.95142,300.9,296.4,17.94,77.36857,22.82,15.372857,2.3714285,26.714285,6.3714285,31.7,22.2,8.6,5.0,2
1990.0,20.0,1990-05-14,0.03225,0.1729667,0.1572,0.1708429,34.54,298.78143,298.87857,295.4343,300.5,297.3,26.1,82.05286,34.54,16.848572,2.3,26.714285,6.4857144,32.2,22.8,41.4,4.0,2
1990.0,21.0,1990-05-21,0.1286333,0.2450667,0.2275571,0.2358857,15.36,298.98715,299.22858,295.31,301.4,297.0,13.9,80.33714,15.36,16.672857,2.4285715,27.471428,6.7714286,33.3,23.3,4.0,3.0,2
1990.0,22.0,1990-05-28,0.1962,0.2622,0.2512,0.24734,7.52,299.5186,299.66428,295.82144,301.9,297.5,12.2,80.46,7.52,17.21,3.0142858,28.942858,9.3714285,35.0,23.9,5.8,6.0,2
1990.0,23.0,1990-06-04,null,0.17485,0.2543143,0.1817429,9.58,299.63,299.76428,295.85144,302.4,298.1,26.49,79.891426,9.58,17.212856,2.1,28.114286,6.9428573,34.4,23.9,39.1,2.0,2
1990.0,24.0,1990-06-11,0.1129,0.0928,0.2050714,0.2102714,3.48,299.20715,299.22144,295.86572,301.3,297.7,38.6,82.0,3.48,17.234285,2.0428572,27.414286,6.7714286,32.2,23.3,29.7,4.0,2
1990.0,25.0,1990-06-18,0.0725,0.0725,0.1514714,0.1330286,151.12,299.59143,299.52856,296.53143,300.6,298.4,30.0,83.37572,151.12,17.977142,1.5714285,28.37143,7.6857142,33.9,22.8,21.1,5.0,2
1990.0,26.0,1990-06-25,0.10245,0.146175,0.1255714,0.1236,19.32,299.57858,299.55713,296.37857,302.1,297.7,37.51,82.76857,19.32,17.79,1.8857143,28.328571,7.385714,33.9,22.8,21.1,10.0,2
1990.0,27.0,1990-07-02,null,0.12155,0.1606833,0.2025667,14.41,300.1543,300.27856,296.65143,302.3,298.7,28.4,81.281425,14.41,18.071428,2.0142858,28.328571,6.5142856,33.9,24.4,1.1,6.0,2


In [9]:
train_labels_transform_city_year = train_labels_transform_city.withColumn('year', substring(col('week_start_date'),0,4))
train_labels_transform_city_year_month = train_labels_transform_city_year.withColumn('month', substring(col('week_start_date'), 6,2))
train_labels_transform_city_year_month_day = train_labels_transform_city_year_month.withColumn('day', substring(col('week_start_date'), 9,2))

train_clean = train_labels_transform_city_year_month_day.drop('week_start_date')
display(train_clean)

year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,num_city,month,day
1990,18.0,0.1226,0.103725,0.1984833,0.1776167,12.42,297.57285,297.74286,292.41428,299.8,295.9,32.0,73.365715,12.42,14.012857,2.6285715,25.442858,6.9,29.4,20.0,16.0,4.0,2,04,30
1990,19.0,0.1699,0.142175,0.1623571,0.1554857,22.82,298.21143,298.44287,293.95142,300.9,296.4,17.94,77.36857,22.82,15.372857,2.3714285,26.714285,6.3714285,31.7,22.2,8.6,5.0,2,05,07
1990,20.0,0.03225,0.1729667,0.1572,0.1708429,34.54,298.78143,298.87857,295.4343,300.5,297.3,26.1,82.05286,34.54,16.848572,2.3,26.714285,6.4857144,32.2,22.8,41.4,4.0,2,05,14
1990,21.0,0.1286333,0.2450667,0.2275571,0.2358857,15.36,298.98715,299.22858,295.31,301.4,297.0,13.9,80.33714,15.36,16.672857,2.4285715,27.471428,6.7714286,33.3,23.3,4.0,3.0,2,05,21
1990,22.0,0.1962,0.2622,0.2512,0.24734,7.52,299.5186,299.66428,295.82144,301.9,297.5,12.2,80.46,7.52,17.21,3.0142858,28.942858,9.3714285,35.0,23.9,5.8,6.0,2,05,28
1990,23.0,null,0.17485,0.2543143,0.1817429,9.58,299.63,299.76428,295.85144,302.4,298.1,26.49,79.891426,9.58,17.212856,2.1,28.114286,6.9428573,34.4,23.9,39.1,2.0,2,06,04
1990,24.0,0.1129,0.0928,0.2050714,0.2102714,3.48,299.20715,299.22144,295.86572,301.3,297.7,38.6,82.0,3.48,17.234285,2.0428572,27.414286,6.7714286,32.2,23.3,29.7,4.0,2,06,11
1990,25.0,0.0725,0.0725,0.1514714,0.1330286,151.12,299.59143,299.52856,296.53143,300.6,298.4,30.0,83.37572,151.12,17.977142,1.5714285,28.37143,7.6857142,33.9,22.8,21.1,5.0,2,06,18
1990,26.0,0.10245,0.146175,0.1255714,0.1236,19.32,299.57858,299.55713,296.37857,302.1,297.7,37.51,82.76857,19.32,17.79,1.8857143,28.328571,7.385714,33.9,22.8,21.1,10.0,2,06,25
1990,27.0,null,0.12155,0.1606833,0.2025667,14.41,300.1543,300.27856,296.65143,302.3,298.7,28.4,81.281425,14.41,18.071428,2.0142858,28.328571,6.5142856,33.9,24.4,1.1,6.0,2,07,02


In [10]:
def fill_with_mean(df, exclude=set()): 
    stats = df.agg(*(
        avg(c).alias(c) for c in df.columns if c not in exclude
    ))
    return df.na.fill(stats.first().asDict())

train_clean_nonull = fill_with_mean(train_clean, ["year", "month", "day" , "num_city"])

In [11]:
describe_mainTrain_df = train_clean_nonull.describe()

## Normalization 
  - x = value 
  - dl = min of attribute 
  - dh = max of attribute 
  - nl = min of expected range 
  - nh = max of expected range

In [13]:
mainTrain = train_clean_nonull

In [14]:
# call function
#normalize columns
def normalizing_column_1(c , dL, dH):
  nL = 0
  nH = 1
  numi = (float(c) - dL) * (nH-nL)
  denom = dH - dL
  div = float(numi) / float(denom)
  normalized = float(div + nL)
  return normalized

normalizing_column = udf(normalizing_column_1, DoubleType())


names = mainTrain.schema.names
for colname in names:
  dL = float(describe_mainTrain_df.collect()[3][colname])
  dH = float(describe_mainTrain_df.collect()[4][colname])
  mainTrain = mainTrain.withColumn('normalized_' + str(colname), 
                           normalizing_column(colname, lit(dL) , lit(dH))
                          )                                                                   
    

In [15]:
normalized_mainTrain = spark.read.parquet("s3a://dengi-ghazalg/normalized_mainTrain2_24NOV2017")
display(normalized_mainTrain)

year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,num_city,month,day,normalized_year,normalized_weekofyear,normalized_ndvi_ne,normalized_ndvi_nw,normalized_ndvi_se,normalized_ndvi_sw,normalized_precipitation_amt_mm,normalized_reanalysis_air_temp_k,normalized_reanalysis_avg_temp_k,normalized_reanalysis_dew_point_temp_k,normalized_reanalysis_max_air_temp_k,normalized_reanalysis_min_air_temp_k,normalized_reanalysis_precip_amt_kg_per_m2,normalized_reanalysis_relative_humidity_percent,normalized_reanalysis_sat_precip_amt_mm,normalized_reanalysis_specific_humidity_g_per_kg,normalized_reanalysis_tdtr_k,normalized_station_avg_temp_c,normalized_station_diur_temp_rng_c,normalized_station_max_temp_c,normalized_station_min_temp_c,normalized_station_precip_mm,normalized_total_cases,normalized_num_city,normalized_month,normalized_day
1990,18.0,0.1226,0.103725,0.1984833,0.1776167,12.42,297.57285,297.74286,292.41428,299.8,295.9,32.0,73.365715,12.42,14.012857,2.6285715,25.442858,6.9,29.4,20.0,16.0,4.0,2,04,30,0.0,0.3269230769230769,0.5782264281858047,0.6148351641188807,0.3864178911251776,0.39554419682730296,0.03179723521836647,0.38829045106412863,0.3546679617256897,0.3146791099330512,0.12345603660300865,0.6923072228064922,0.056091148115687994,0.38161394261233145,0.03179723521836647,0.2626593365771838,0.08666018102141959,0.43009124918186925,0.21039289885986687,0.1741935237761467,0.4862385321100917,0.029449659488312168,0.008676789587852495,1.0,0.2727272727272727,1.0
1990,19.0,0.1699,0.142175,0.1623571,0.1554857,22.82,298.21143,298.44287,293.95142,300.9,296.4,17.94,77.36857,22.82,15.372857,2.3714285,26.714285,6.3714285,31.7,22.2,8.6,5.0,2,05,07,0.0,0.34615384615384615,0.6299426279277379,0.6570633848142966,0.3211902103920671,0.35923255465928833,0.058422938286800355,0.47271073083431553,0.4417806383941173,0.4892122895176079,0.1913576479311337,0.7307687612680306,0.03144610084847961,0.47966816482438557,0.058422938286800355,0.41816403941531066,0.06913339360865119,0.5653494571117644,0.1634980788845767,0.3225806943831905,0.6880734644898581,0.015829192677102387,0.010845986984815618,1.0,0.36363636363636365,0.20689655172413793
1990,20.0,0.03225,0.1729667,0.1572,0.1708429,34.54,298.78143,298.87857,295.4343,300.5,297.3,26.1,82.05286,34.54,16.848572,2.3,26.714285,6.4857144,32.2,22.8,41.4,4.0,2,05,14,0.0,0.36538461538461536,0.47944084219471683,0.6908807738412004,0.3118787810459555,0.3844300206096303,0.08842806173970133,0.5480656644327648,0.4960010449158342,0.6575845369156741,0.16666666666666607,0.7999990609975979,0.045749343350516614,0.5944148651753445,0.08842806173970133,0.5868997977001934,0.06426484696639398,0.5653494571117644,0.17363751657042795,0.35483875889931954,0.7431191960606005,0.07620099673454613,0.008676789587852495,1.0,0.36363636363636365,0.4482758620689655
1990,21.0,0.1286333,0.2450667,0.2275571,0.2358857,15.36,298.98715,299.22858,295.31,301.4,297.0,13.9,80.33714,15.36,16.672857,2.4285715,27.471428,6.7714286,33.3,23.3,4.0,3.0,2,05,21,0.0,0.38461538461538464,0.5848230408777849,0.7700655447460686,0.4389120937559028,0.49114960311174877,0.039324115864509075,0.5752617029479777,0.539557383250048,0.6434712203827292,0.2222218454619979,0.7769230769230787,0.02436459179409338,0.5523866077458162,0.039324115864509075,0.566808299990818,0.07302824067277819,0.6458965869660073,0.19898604732763858,0.425806402391003,0.7889907556936281,0.007362414872078042,0.006507592190889371,1.0,0.36363636363636365,0.6896551724137931
1990,22.0,0.1962,0.2622,0.2512,0.24734,7.52,299.5186,299.66428,295.82144,301.9,297.5,12.2,80.46,7.52,17.21,3.0142858,28.942858,9.3714285,35.0,23.9,5.8,

In [16]:
normalized_mainTest = spark.read.parquet("s3a://dengi-ghazalg/normalized_mainTest2_24NOV2017")
display(normalized_mainTest)

year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,num_city,month,day,normalized_year,normalized_weekofyear,normalized_ndvi_ne,normalized_ndvi_nw,normalized_ndvi_se,normalized_ndvi_sw,normalized_precipitation_amt_mm,normalized_reanalysis_air_temp_k,normalized_reanalysis_avg_temp_k,normalized_reanalysis_dew_point_temp_k,normalized_reanalysis_max_air_temp_k,normalized_reanalysis_min_air_temp_k,normalized_reanalysis_precip_amt_kg_per_m2,normalized_reanalysis_relative_humidity_percent,normalized_reanalysis_sat_precip_amt_mm,normalized_reanalysis_specific_humidity_g_per_kg,normalized_reanalysis_tdtr_k,normalized_station_avg_temp_c,normalized_station_diur_temp_rng_c,normalized_station_max_temp_c,normalized_station_min_temp_c,normalized_station_precip_mm,normalized_num_city,normalized_month,normalized_day
2008,18.0,-0.0189,-0.0189,0.1027286,0.0912,78.6,298.49286,298.55,294.52713,301.1,296.4,25.37,78.781425,78.6,15.918571,3.1285715,26.528572,7.0571427,33.3,21.7,75.2,2,04,29,0.0,0.3269230769230769,0.46119526918559284,0.22409386668340422,0.21602356850652688,0.19471885454591537,0.4641549455186081,0.5335788450319379,0.40952985631392935,0.5316390914979416,0.1823903209758258,0.7555551034432879,0.08417385812618912,0.419244598805857,0.4641549455186081,0.4788589493950825,0.12637363447900243,0.38785046932202083,0.2821798710733968,0.5446427890232632,0.6000000610351564,0.35471696673699143,1.0,0.2727272727272727,0.9655172413793104
2008,19.0,-0.018,-0.0124,0.08204286,0.07231429,12.56,298.4757,298.55713,294.39572,300.8,296.7,21.83,78.23,12.56,15.791429,2.5714285,26.071428,5.5571427,30.0,22.2,34.3,2,05,06,0.0,0.34615384615384615,0.46212907322562813,0.2316449809989106,0.16973047575461087,0.15998423692908573,0.0741703107335343,0.5312553184778627,0.4104122530539261,0.5128010533045176,0.16352124484080224,0.7777786820023157,0.0724286659711548,0.4025666066678319,0.0741703107335343,0.46085368357863593,0.08351648172490146,0.31308397838541024,0.14175860915993993,0.25000000000000006,0.6400000610351563,0.16179244923141767,1.0,0.36363636363636365,0.1724137931034483
2008,20.0,-0.0015,0.12680304,0.1510833,0.09152857,3.66,299.45572,299.35715,295.30856,302.2,296.4,4.12,78.27,3.66,16.674286,4.428571,27.928572,7.785714,32.8,22.8,3.0,2,05,13,0.0,0.36538461538461536,0.47924880679286314,0.3933585507719047,0.3242376805765426,0.19532316196654992,0.02161332281699946,0.6640229487821583,0.5092670844172361,0.6436611086788094,0.2515730947818398,0.7555551034432879,0.01366954175699762,0.40377622029874016,0.02161332281699946,0.5858789897444093,0.22637361477882548,0.6168223214006482,0.35038448272739825,0.4999999318804061,0.6879999389648438,0.014150943396226415,1.0,0.36363636363636365,0.41379310344827586
2008,21.0,0.12604976,-0.01986667,0.1243286,0.1256857,0.0,299.69,299.72858,294.40286,303.0,296.9,2.2,73.01572,0.0,15.775714,4.3428574,28.057142,6.2714286,33.3,24.4,0.3,2,05,20,0.0,0.38461538461538464,0.6115892892755594,0.22297087645151245,0.2643627064010389,0.2581450018701131,0.0,0.6957626522619352,0.5551630276757846,0.5138247646229712,0.30188679245283023,0.7925921404803249,0.007299270231200119,0.24485832601094923,0.0,0.4586282635700594,0.21978024052107048,0.6378502117565679,0.20862588949169567,0.5446427890232632,0.8159999694824219,0.0014150943958534386,1.0,0.36363636363636365,0.6551724137931034
2008,22.0,0.0568,0.03983333,0.06226667,0.07591429,0.76,299.78,299.67142,294.76,302.3,297.3,4.36,74.08428,0.76,16.137142,3.5428572,27.614286,7.0857143,33.3,23.3,84.1,2,05,27,0.0,0.40384615384615385,0.5397385353491895,0.2923249647048991,0.1254728902874951,0.166605363385591,0.004488012226663853,0.70795496508

In [17]:
normalized_mainTest.select('normalized_num_city').groupby('normalized_num_city').count().show()

+-------------------+-----+
normalized_num_city|count|
+-------------------+-----+
 0.0| 156|
 1.0| 260|
+-------------------+-----+

## Tensor Flow

In [19]:
import pandas as pd
import numpy as np

In [20]:
normalized_mainTrain_pd = normalized_mainTrain.toPandas()
normalized_mainTest_pd = normalized_mainTest.toPandas()

In [21]:
normalized_mainTest_pd.year = normalized_mainTest_pd.year.astype(float)

In [22]:
non_feature_columns = ['year' , 'weekofyear']

In [23]:
feature_columns = [
  'normalized_year', 
  'normalized_weekofyear', 
  'normalized_ndvi_ne', 
  'normalized_ndvi_nw',
  'normalized_ndvi_se', 
  'normalized_ndvi_sw', 
  'normalized_precipitation_amt_mm',
  'normalized_reanalysis_air_temp_k', 
  'normalized_reanalysis_avg_temp_k',
  'normalized_reanalysis_dew_point_temp_k', 
  'normalized_reanalysis_max_air_temp_k',
  'normalized_reanalysis_min_air_temp_k',
  'normalized_reanalysis_precip_amt_kg_per_m2',
  'normalized_reanalysis_relative_humidity_percent',
  'normalized_reanalysis_sat_precip_amt_mm',
  'normalized_reanalysis_specific_humidity_g_per_kg', 
  'normalized_reanalysis_tdtr_k',
  'normalized_station_avg_temp_c', 
  'normalized_station_diur_temp_rng_c',
  'normalized_station_max_temp_c', 
  'normalized_station_min_temp_c',
  'normalized_station_precip_mm', 
  'normalized_num_city',
  'normalized_month',
  'normalized_day'
]

label_columns = ['total_cases']

In [24]:
import tensorflow as tf

tf.reset_default_graph()

feature_columns_tf = [
  tf.feature_column.numeric_column("normalized_year"), 
  tf.feature_column.numeric_column("normalized_weekofyear"), 
  tf.feature_column.numeric_column("normalized_ndvi_ne"), 
  tf.feature_column.numeric_column("normalized_ndvi_nw"),
  tf.feature_column.numeric_column("normalized_ndvi_se"), 
  tf.feature_column.numeric_column("normalized_ndvi_sw"), 
  tf.feature_column.numeric_column("normalized_precipitation_amt_mm"),
  tf.feature_column.numeric_column("normalized_reanalysis_air_temp_k"), 
  tf.feature_column.numeric_column("normalized_reanalysis_avg_temp_k"),
  tf.feature_column.numeric_column("normalized_reanalysis_dew_point_temp_k"), 
  tf.feature_column.numeric_column("normalized_reanalysis_max_air_temp_k"),
  tf.feature_column.numeric_column("normalized_reanalysis_min_air_temp_k"),
  tf.feature_column.numeric_column("normalized_reanalysis_precip_amt_kg_per_m2"),
  tf.feature_column.numeric_column("normalized_reanalysis_relative_humidity_percent"),
  tf.feature_column.numeric_column("normalized_reanalysis_sat_precip_amt_mm"),
  tf.feature_column.numeric_column("normalized_reanalysis_specific_humidity_g_per_kg"), 
  tf.feature_column.numeric_column("normalized_reanalysis_tdtr_k"),
  tf.feature_column.numeric_column("normalized_station_avg_temp_c"), 
  tf.feature_column.numeric_column("normalized_station_diur_temp_rng_c"),
  tf.feature_column.numeric_column("normalized_station_max_temp_c"), 
  tf.feature_column.numeric_column("normalized_station_min_temp_c"),
  tf.feature_column.numeric_column("normalized_station_precip_mm"), 
  tf.feature_column.numeric_column("normalized_num_city"),
  tf.feature_column.numeric_column("normalized_month"),
  tf.feature_column.numeric_column("normalized_day")
]

# Define the train inputs
train_input_fn = tf.estimator.inputs.pandas_input_fn(
    x = normalized_mainTrain_pd[feature_columns],
    y = normalized_mainTrain_pd[label_columns],
    num_epochs=None,
    shuffle=True)




# Define train inputs for evaluation
train_input_fn_eval = tf.estimator.inputs.pandas_input_fn(
    x = normalized_mainTrain_pd[feature_columns],
    y = normalized_mainTrain_pd[label_columns],
    num_epochs = 1,
    shuffle = False)


# Define test inputs for evaluation
test_input_fn_eval = tf.estimator.inputs.pandas_input_fn(
    x = normalized_mainTest_pd[feature_columns],
    y = None,
    num_epochs=1,
    shuffle=False)



classifier = tf.estimator.DNNRegressor(
  feature_columns = feature_columns_tf,
  hidden_units = [200, 300, 200, 300, 200, 300],
  optimizer = tf.train.ProximalAdagradOptimizer(
    learning_rate=0.01,
    l1_regularization_strength=0.001
  ),
  activation_fn = tf.nn.relu,
  model_dir= "/tmp/tf_dengai_mainTrain_ghazall_24Nov_v3"
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_model_dir': '/tmp/tf_dengai_mainTrain_ghazall_24Nov_v3', '_save_summary_steps': 100}

In [25]:
# Train model.
classifier.train(input_fn = train_input_fn, steps=50000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tf_dengai_mainTrain_ghazall_24Nov_v3/model.ckpt.
INFO:tensorflow:loss = 124097.0, step = 1
INFO:tensorflow:global_step/sec: 61.3542
INFO:tensorflow:loss = 50551.6, step = 101 (1.630 sec)
INFO:tensorflow:global_step/sec: 58.0391
INFO:tensorflow:loss = 64033.1, step = 201 (1.723 sec)
INFO:tensorflow:global_step/sec: 61.6817
INFO:tensorflow:loss = 174109.0, step = 301 (1.621 sec)
INFO:tensorflow:global_step/sec: 64.3861
INFO:tensorflow:loss = 382284.0, step = 401 (1.553 sec)
INFO:tensorflow:global_step/sec: 63.4465
INFO:tensorflow:loss = 103059.0, step = 501 (1.576 sec)
INFO:tensorflow:global_step/sec: 63.4738
INFO:tensorflow:loss = 35550.5, step = 601 (1.575 sec)
INFO:tensorflow:global_step/sec: 63.0768
INFO:tensorflow:loss = 147656.0, step = 701 (1.585 sec)
INFO:tensorflow:global_step/sec: 61.5127
INFO:tensorflow:loss = 291569.0, step = 801 (1.626 sec)
INFO:tensorflow:global_step/sec: 62.3735
INFO:tensorflow:loss = 160285.0, step = 901 (1.603 sec)
INFO:tensorflow:global_step/sec: 62.4334
INFO:tensorflow:loss = 129315.0, step = 1001 (1.602 sec)
INFO:tensorflow:global_step/sec: 60.464
INFO:tensorflow:loss = 121251.0, step = 1101 (1.654 sec)
INFO:tensorflow:global_step/sec: 63.2063
INFO:tensorflow:loss = 50971.6, step = 1201 (1.582 sec)
INFO:tensorflow:global_step/sec: 62.8762
INFO:tensorflow:loss = 193034.0, step = 1301 (1.590 sec)
INFO:tensorflow:global_step/sec: 61.0859
INFO:tensorflow:loss = 202149.0, step = 1401 (1.637 sec)
INFO:tensorflow:global_step/sec: 62.1911
INFO:tensorflow:loss = 50781.1, step = 1501 (1.608 sec)
INFO:tensorflow:global_step/sec: 61.0662
INFO:tensorflow:loss = 27850.4, step = 1601 (1.638 sec)
INFO:tensorflow:global_step/sec: 62.2449
INFO:tensorflow:loss = 49568.0, step = 1701 (1.606 sec)
INFO:tensorflow:global_step/sec: 62.1225
INFO:tensorflow:loss = 102440.0, step = 1801 (1.610 sec)
INFO:tensorflow:global_step/sec: 62.9703
INFO:tensorflow:loss = 80016.4, step = 1901 (1.588 sec)
INFO:tensorflow:global_step/sec: 62.215
INFO:tensorflow:loss = 35907.0, step = 2001 (1.607 sec)
INFO:tensorflow:global_step/sec: 59.7121
INFO:tensorflow:loss = 33715.3, step = 2101 (1.675 sec)
INFO:tensorflow:global_step/sec: 63.2643
INFO:tensorflow:loss = 51102.9, step = 2201 (1.581 sec)
INFO:tensorflow:global_step/sec: 62.7859
INFO:tensorflow:loss = 35571.1, step = 2301 (1.593 sec)
INFO:tensorflow:global_step/sec: 62.8598
INFO:tensorflow:loss = 27902.8, step = 2401 (1.591 sec)
INFO:tensorflow:global_step/sec: 68.3808
INFO:tensorflow:loss = 33343.2, step = 2501 (1.462 sec)
INFO:tensorflow:global_step/sec: 60.7581
INFO:tensorflow:loss = 15521.6, step = 2601 (1.646 sec)
INFO:tensorflow:global_step/sec: 61.8274
INFO:tensorflow:loss = 18307.7, step = 2701 (1.618 sec)
INFO:tensorflow:global_step/sec: 63.2403
INFO:tensorflow:loss = 16213.3, step = 2801 (1.581 sec)
INFO:tensorflow:global_step/sec: 61.9179
INFO:tensorflow:loss = 158778.0, step = 2901 (1.615 sec)
INFO:tensorflow:global_step/sec: 60.1557
INFO:tensorflow:loss = 88333.4, step = 3001 (1.662 sec)
INFO:tensorflow:global_step/sec: 61.6199
INFO:tensorflow:loss = 29715.6, step = 3101 (1.623 sec)
INFO:tensorflow:global_step/sec: 63.1234
INFO:tensorflow:loss = 33888.8, step = 3201 (1.584 sec)
INFO:tensorflow:global_step/sec: 61.5726
INFO:tensorflow:loss = 21561.2, step = 3301 (1.624 sec)
INFO:tensorflow:global_step/sec: 62.5453
INFO:tensorflow:loss = 22791.3, step = 3401 (1.599 sec)
INFO:tensorflow:global_step/sec: 62.4165
INFO:tensorflow:loss = 11895.6, step = 3501 (1.602 sec)
INFO:tensorflow:global_step/sec: 63.8913
INFO:tensorflow:loss = 12773.7, step = 3601 (1.565 sec)
INFO:tensorflow:global_step/sec: 63.6485
INFO:tensorflow:loss = 12782.7, step = 3701 (1.571 sec)
INFO:tensorflow:global_step/sec: 64.3587
INFO:tensorflow:loss = 13354.8, step = 3801 (1.554 sec)
INFO:tensorflow:global_step/sec: 61.3272
INFO:tensorflow:loss = 13567.5, step = 3901 (1.631 sec)
INFO:tensorflow:global_step/se

In [26]:
# Evaluate accuracy.
average_loss_train = classifier.evaluate(input_fn = train_input_fn_eval)["average_loss"]
print("Train Average Loss: {0:f}\n".format(average_loss_train))

INFO:tensorflow:Starting evaluation at 2017-11-24-10:41:57
INFO:tensorflow:Restoring parameters from /tmp/tf_dengai_mainTrain_ghazall_24Nov_v3/model.ckpt-50000
INFO:tensorflow:Finished evaluation at 2017-11-24-10:41:57
INFO:tensorflow:Saving dict for global step 50000: average_loss = 0.210588, global_step = 50000, loss = 25.5514
Train Average Loss: 0.210588

In [27]:
# Evaluate accuracy.
average_loss_test = classifier.predict(input_fn = test_input_fn_eval)
#print("Test Average Loss: {0:f}\n".format(average_loss_test))

In [28]:
predictions_list = [prediction['predictions'][0] for prediction in average_loss_test]
len(predictions_list)

INFO:tensorflow:Restoring parameters from /tmp/tf_dengai_mainTrain_ghazall_24Nov_v3/model.ckpt-50000
 Out[ 29 ]: 416

In [29]:
print predictions_list

[6.5850163, 35.382427, 28.507847, 29.418766, 6.3669591, 25.282085, 2.4309013, 26.461126, 12.803143, 22.841993, 17.440418, 13.611083, 3.4472148, 39.81982, 18.41518, 31.858532, 1.77414, 65.331619, 17.26852, 6.3350949, 55.840069, 3.1736066, 15.836232, 95.563889, 86.650726, 24.525089, 112.6334, 44.118809, 73.959282, 81.777504, 131.46739, 69.517723, 26.33873, 21.350235, 80.114479, 52.845181, 92.631615, 91.714523, 71.784996, 130.52618, 81.172035, 47.270432, 138.58134, 70.32711, 40.776737, 41.418503, 66.770393, 86.188728, 9.6839256, 7.0955968, 14.703615, 11.7488, 11.473741, 14.910182, 12.890844, 34.920025, 11.733029, 6.1269722, 13.474046, 11.183696, 26.630329, 15.455327, 11.292247, 14.516002, 26.898577, 83.895149, 12.060964, 22.838074, 34.817097, 56.968582, 42.891148, 47.149109, 39.344238, 53.744267, 91.280312, 108.99261, 117.34008, 12.320558, 43.447769, 75.253487, 29.663174, 36.180786, 33.721912, 58.22308, 67.215607, 85.716515, 65.643288, 83.13044, 41.404308, 85.117195, 7.73739, 248.05586, 28.157148, 44.331158, 85.776833, 29.462231, 100.8504, 60.802471, 59.286224, 18.760525, 17.078478, 8.490407, 9.3597021, 24.479425, 4.494565, 20.777437, 57.775581, 232.5045, 116.59955, 26.054142, 141.23695, 196.60233, 268.67755, 170.87184, 307.77771, 176.6281, 71.220764, 25.651615, 217.77869, 181.51082, 284.08301, 177.82347, 334.88696, 62.953701, 312.63846, 84.338364, 120.51063, 84.537209, 32.814228, 18.968761, 2.5699153, 32.348026, 41.655029, 79.800499, 5.6469412, 15.431476, 0.90614784, 14.940019, 16.2337, 9.5337801, 11.842875, 21.808729, 54.122646, 46.82893, 15.144938, 7.6984, 26.408079, 24.861073, 3.6725566, 5.3316889, 6.606111, 6.1972566, 7.8720212, 4.1292586, 3.300602, 8.6815062, 5.4277339, 8.6401663, 5.8536916, 30.339819, 119.78713, 6.3546448, 3.7324898, 417.11365, 247.40784, 28.122753, 39.883629, 26.152109, 71.314476, 139.1422, 133.14029, 79.083519, 6.4035945, 56.392506, 7.5091996, 18.092131, 34.758064, 36.438961, 14.513366, -0.38776854, 20.079067, 3.9887426, 9.2986479, 18.313797, 24.306213, 20.334377, 44.78149, 17.203297, 4.5188828, 33.197735, 9.1689482, 16.174618, 12.914039, 19.745117, 13.22338, 18.315857, 5.1048365, 9.3693476, 11.203939, 10.974353, 4.6665769, 2.5968754, 3.4486361, 6.579422, 2.6733561, 1.6729325, 5.5871239, 9.9741831, 6.8537564, 6.416235, 6.4582138, 6.3772521, 5.341846, 9.9430265, 9.3083544, 8.6057158, 26.654696, 4.9457297, 4.4463954, 40.010487, 14.470019, 6.8829436, 35.371716, 36.621834, 53.282597, 18.828545, 7.2338414, 43.779629, 28.524681, 4.2765613, 10.556331, 62.331676, 38.411747, 1.173642, 6.8766155, 9.391449, 33.871883, 7.4767518, 16.974426, 7.5839996, 4.9644303, 18.194733, 16.610031, 4.8488455, 11.930248, 8.1283731, 12.599088, 16.48934, 5.4878492, 6.3007731, 10.728837, 8.3225536, 14.000954, 5.107553, 4.3599362, 14.281561, 3.7059033, 4.4088497, 4.303534, 6.4096618, 0.54490548, 0.04124482, 0.24659704, 0.38791311, 8.6291227, 0.095910504, 0.70165044, -0.082587585, 0.042226031, -0.0081278514, 0.44998592, 0.14006539, 0.68423945, 0.39581856, 1.1430048, 0.59375995, 0.92447257, 0.75776047, 0.97846466, 0.7854262, 0.97679681, 0.83505553, 0.13019539, 1.1582382, 0.72959971, 0.048529103, 1.1147557, 51.562603, 1.1365796, 2.2907403, 2.0439975, 1.9548285, 39.038296, 1.1467808, 8.3037176, 25.001635, 14.664443, 7.7581968, 1.0102223, 37.033413, 9.3536882, 5.6981854, 1.3044671, 0.74018288, 9.5512037, 0.46795145, 15.279497, 13.529408, -0.19200872, 0.54284233, 0.43203673, 0.56302613, -1.4087111, -1.8998395, 0.72459346, 3.7002409, 18.715282, 0.22360803, -0.0035894047, 0.7035296, 0.62169451, -0.23034792, 0.33912, 3.3996351, 0.57273513, -0.70548707, 0.36553633, 0.42110464, 0.2323616, 0.38934132, 0.38064721, 0.25480434, 2.8862484, 1.2163755, 1.228663, 2.021208, -0.33344343, 1.137715, 13.630555, 4.6573663, 9.703248, 6.7305017, 9.8468437, 15.604887, 26.274578, 10.05574, 7.2884183, 7.448936, 16.933784, 5.8563004, 0.10856967, 40.177708, 6.689836, 3.1997764, 7.3578734, 4.9148388, 5.2293539, 6.9272127, 11.56881, 0.89237505, 1.4511454, 3.5061936, 29

In [30]:
list_to_df = pd.DataFrame({'predictions':predictions_list})
print (list_to_df)

predictions
0 6.585016
1 35.382427
2 28.507847
3 29.418766
4 6.366959
5 25.282085
6 2.430901
7 26.461126
8 12.803143
9 22.841993
10 17.440418
11 13.611083
12 3.447215
13 39.819820
14 18.415180
15 31.858532
16 1.774140
17 65.331619
18 17.268520
19 6.335095
20 55.840069
21 3.173607
22 15.836232
23 95.563889
24 86.650726
25 24.525089
26 112.633400
27 44.118809
28 73.959282
29 81.777504
.. ...
386 1.583188
387 7.457901
388 30.319458
389 1.661356
390 1.769947
391 4.551986
392 16.554348
393 26.920647
394 36.413830
395 33.323650
396 31.150530
397 17.227360
398 11.142330
399 3.661246
400 6.109979
401 12.189935
402 15.402799
403 27.202997
404 6.175995
405 9.192692
406 4.676996
407 6.917217
408 2.502229
409 2.836050
410 9.633609
411 5.365300
412 3.780128
413 3.756881
414 1.649735
415 3.022976

[416 rows x 1 columns]

In [31]:
cons = normalized_mainTest_pd[feature_columns].join(list_to_df)

In [32]:
spark.createDataFrame(
    cons
  ).count()

Out[ 34 ]: 416

In [33]:
import math

In [34]:
def roundup(value):
  return math.ceil(value)

roundup = udf(roundup,FloatType())

def str_city(city):
  if(city == 1):
    return 'sj'
  elif(city == 2):
    return 'iq'

str_city = udf(str_city,StringType())

In [35]:
normalized_mainTest_pd.year = normalized_mainTest_pd.year.astype(float)

In [36]:
normalized_mainTest_pd_selected = normalized_mainTest_pd[['year', 'weekofyear']]


In [37]:
normalized_mainTest_pd_selected

Out[ 40 ]: 
 year weekofyear
0 2008.0 18.0
1 2008.0 19.0
2 2008.0 20.0
3 2008.0 21.0
4 2008.0 22.0
5 2008.0 23.0
6 2008.0 24.0
7 2008.0 25.0
8 2008.0 26.0
9 2008.0 27.0
10 2008.0 28.0
11 2008.0 29.0
12 2008.0 30.0
13 2008.0 31.0
14 2008.0 32.0
15 2008.0 33.0
16 2008.0 34.0
17 2008.0 35.0
18 2008.0 36.0
19 2008.0 37.0
20 2008.0 38.0
21 2008.0 39.0
22 2008.0 40.0
23 2008.0 41.0
24 2008.0 42.0
25 2008.0 43.0
26 2008.0 44.0
27 2008.0 45.0
28 2008.0 46.0
29 2008.0 47.0
.. ... ...
386 2012.0 48.0
387 2012.0 49.0
388 2012.0 50.0
389 2012.0 51.0
390 2013.0 1.0
391 2013.0 2.0
392 2013.0 3.0
393 2013.0 4.0
394 2013.0 5.0
395 2013.0 6.0
396 2013.0 7.0
397 2013.0 8.0
398 2013.0 9.0
399 2013.0 10.0
400 2013.0 11.0
401 2013.0 12.0
402 2013.0 13.0
403 2013.0 14.0
404 2013.0 15.0
405 2013.0 16.0
406 2013.0 17.0
407 2013.0 18.0
408 2013.0 19.0
409 2013.0 20.0
410 2013.0 21.0
411 2013.0 22.0
412 2013.0 23.0
413 2013.0 24.0
414 2013.0 25.0
415 2013.0 26.0

[416 rows x 2 columns]

In [38]:
result = pd.concat([cons, normalized_mainTest_pd_selected], axis=1)

In [39]:
pd_to_spark = spark.createDataFrame(result)
display(pd_to_spark)

normalized_year,normalized_weekofyear,normalized_ndvi_ne,normalized_ndvi_nw,normalized_ndvi_se,normalized_ndvi_sw,normalized_precipitation_amt_mm,normalized_reanalysis_air_temp_k,normalized_reanalysis_avg_temp_k,normalized_reanalysis_dew_point_temp_k,normalized_reanalysis_max_air_temp_k,normalized_reanalysis_min_air_temp_k,normalized_reanalysis_precip_amt_kg_per_m2,normalized_reanalysis_relative_humidity_percent,normalized_reanalysis_sat_precip_amt_mm,normalized_reanalysis_specific_humidity_g_per_kg,normalized_reanalysis_tdtr_k,normalized_station_avg_temp_c,normalized_station_diur_temp_rng_c,normalized_station_max_temp_c,normalized_station_min_temp_c,normalized_station_precip_mm,normalized_num_city,normalized_month,normalized_day,predictions,year,weekofyear
0.0,0.3269230769230769,0.46119526918559284,0.22409386668340422,0.21602356850652688,0.19471885454591537,0.4641549455186081,0.5335788450319379,0.40952985631392935,0.5316390914979416,0.1823903209758258,0.7555551034432879,0.08417385812618912,0.419244598805857,0.4641549455186081,0.4788589493950825,0.12637363447900243,0.38785046932202083,0.2821798710733968,0.5446427890232632,0.6000000610351564,0.35471696673699143,1.0,0.2727272727272727,0.9655172413793104,6.585016250610352,2008.0,18.0
0.0,0.34615384615384615,0.46212907322562813,0.2316449809989106,0.16973047575461087,0.15998423692908573,0.0741703107335343,0.5312553184778627,0.4104122530539261,0.5128010533045176,0.16352124484080224,0.7777786820023157,0.0724286659711548,0.4025666066678319,0.0741703107335343,0.46085368357863593,0.08351648172490146,0.31308397838541024,0.14175860915993993,0.25000000000000006,0.6400000610351563,0.16179244923141767,1.0,0.36363636363636365,0.1724137931034483,35.38242721557617,2008.0,19.0
0.0,0.36538461538461536,0.47924880679286314,0.3933585507719047,0.3242376805765426,0.19532316196654992,0.02161332281699946,0.6640229487821583,0.5092670844172361,0.6436611086788094,0.2515730947818398,0.7555551034432879,0.01366954175699762,0.40377622029874016,0.02161332281699946,0.5858789897444093,0.22637361477882548,0.6168223214006482,0.35038448272739825,0.4999999318804061,0.6879999389648438,0.014150943396226415,1.0,0.36363636363636365,0.41379310344827586,28.50784683227539,2008.0,20.0
0.0,0.38461538461538464,0.6115892892755594,0.22297087645151245,0.2643627064010389,0.2581450018701131,0.0,0.6957626522619352,0.5551630276757846,0.5138247646229712,0.30188679245283023,0.7925921404803249,0.007299270231200119,0.24485832601094923,0.0,0.4586282635700594,0.21978024052107048,0.6378502117565679,0.20862588949169567,0.5446427890232632,0.8159999694824219,0.0014150943958534386,1.0,0.36363636363636365,0.6551724137931034,29.418766021728516,2008.0,21.0
0.0,0.40384615384615385,0.5397385353491895,0.2923249647048991,0.1254728902874951,0.166605363385591,0.004488012226663853,0.7079549650875001,0.5481000828295713,0.5650234550497343,0.25786086748231146,0.822221317997686,0.014465826587639034,0.27717753156825264,0.004488012226663853,0.509811708281211,0.15824176289057768,0.5654205343533093,0.28485457835660516,0.5446427890232632,0.7279999389648438,0.39669810601000516,1.0,0.36363636363636365,0.896551724137931,6.366959095001221,2008.0,22.0
0.0,0.4230769230769231,0.4351525215173465,0.21065674933732995,0.2815307108711904,0.1806095506638973,0.420278718371001,0.706408703644041,0.5551630276757846,0.6444835776013277,0.23270401864681625,0.8444448965567138,0.07481751571685649,0.3519702822151218,0.420278718371001,0.5848674351950564,0.10549451207006191,0.6285045173705455,0.10565030634957069,0.4999999318804061,0.8640000000000001,0.13066038095726157,1.0,0.45454545454545453,0.06896551724137931,25.282085418701172,2008.0,23.0
0.0,0.4423076923076923,0.4348412522155978,0.2170945642391379,0.28213807164345184,0.31970562066964486,0.28929964378449746,0.7462766156741945,0.5895840423881361,0.6926067592380357,0.2641505595273193,0.8370370370370379,0.0434638367002977,0.3606549481084572,0.28929964378449746,0.6334209731319023,0.12857143201155116,0.5303736344941403,0.1872283651421799,

In [40]:
new_df = pd_to_spark.select(
   col('normalized_num_city').alias('city'),  
   col('year'),
   col('weekofyear'),
   col('predictions').alias("total_cases")
)

display(new_df)

city,year,weekofyear,total_cases
1.0,2008.0,18.0,6.585016250610352
1.0,2008.0,19.0,35.38242721557617
1.0,2008.0,20.0,28.50784683227539
1.0,2008.0,21.0,29.418766021728516
1.0,2008.0,22.0,6.366959095001221
1.0,2008.0,23.0,25.282085418701172
1.0,2008.0,24.0,2.430901288986206
1.0,2008.0,25.0,26.46112632751465
1.0,2008.0,26.0,12.803142547607422
1.0,2008.0,27.0,22.84199333190918


In [41]:
new_df.select('city').distinct().show()

+----+
city|
+----+
 0.0|
 1.0|
+----+

In [42]:
def str_city(city):
  if(city == 1):
    return 'sj'
  elif(city == 0):
    return 'iq'

str_city = udf(str_city,StringType())

In [43]:
final_df3 = new_df.withColumn("city2", str_city('city')).drop('city')

In [44]:
display(final_df3)

year,weekofyear,total_cases,city2
2008.0,18.0,6.585016250610352,sj
2008.0,19.0,35.38242721557617,sj
2008.0,20.0,28.50784683227539,sj
2008.0,21.0,29.418766021728516,sj
2008.0,22.0,6.366959095001221,sj
2008.0,23.0,25.282085418701172,sj
2008.0,24.0,2.430901288986206,sj
2008.0,25.0,26.46112632751465,sj
2008.0,26.0,12.803142547607422,sj
2008.0,27.0,22.84199333190918,sj


In [45]:
submission_df12 = final_df3.select(
  col('city2').alias('city'),  
  col('year'),
  col('weekofyear'),
  col('total_cases')
)

In [46]:
display(submission_df12)

city,year,weekofyear,total_cases
sj,2008.0,18.0,6.585016250610352
sj,2008.0,19.0,35.38242721557617
sj,2008.0,20.0,28.50784683227539
sj,2008.0,21.0,29.418766021728516
sj,2008.0,22.0,6.366959095001221
sj,2008.0,23.0,25.282085418701172
sj,2008.0,24.0,2.430901288986206
sj,2008.0,25.0,26.46112632751465
sj,2008.0,26.0,12.803142547607422
sj,2008.0,27.0,22.84199333190918


In [47]:
submission_df12.printSchema()

root
-- city: string (nullable = true)
-- year: double (nullable = true)
-- weekofyear: double (nullable = true)
-- total_cases: double (nullable = true)

In [48]:
 import math

In [49]:
def roundup(value):
  return math.ceil(value)

roundup = udf(roundup,FloatType())
submission_df12 = submission_df12.withColumn('total_cases2', roundup('total_cases'))


In [50]:
display(submission_df12)

city,year,weekofyear,total_cases,total_cases2
sj,2008.0,18.0,6.585016250610352,7.0
sj,2008.0,19.0,35.38242721557617,36.0
sj,2008.0,20.0,28.50784683227539,29.0
sj,2008.0,21.0,29.418766021728516,30.0
sj,2008.0,22.0,6.366959095001221,7.0
sj,2008.0,23.0,25.282085418701172,26.0
sj,2008.0,24.0,2.430901288986206,3.0
sj,2008.0,25.0,26.46112632751465,27.0
sj,2008.0,26.0,12.803142547607422,13.0
sj,2008.0,27.0,22.84199333190918,23.0


In [51]:
submission_df6 = submission_df12.drop('total_cases')
display(submission_df6)

city,year,weekofyear,total_cases2
sj,2008.0,18.0,7.0
sj,2008.0,19.0,36.0
sj,2008.0,20.0,29.0
sj,2008.0,21.0,30.0
sj,2008.0,22.0,7.0
sj,2008.0,23.0,26.0
sj,2008.0,24.0,3.0
sj,2008.0,25.0,27.0
sj,2008.0,26.0,13.0
sj,2008.0,27.0,23.0


In [52]:
submission_df7 = submission_df6.select(
  col('city'), 
  col('year'), 
  col('weekofyear'), 
  col('total_cases2').alias('total_cases')
)

In [53]:
display(submission_df7)

city,year,weekofyear,total_cases
sj,2008.0,18.0,7.0
sj,2008.0,19.0,36.0
sj,2008.0,20.0,29.0
sj,2008.0,21.0,30.0
sj,2008.0,22.0,7.0
sj,2008.0,23.0,26.0
sj,2008.0,24.0,3.0
sj,2008.0,25.0,27.0
sj,2008.0,26.0,13.0
sj,2008.0,27.0,23.0


In [54]:
submission_df7.printSchema()

root
-- city: string (nullable = true)
-- year: double (nullable = true)
-- weekofyear: double (nullable = true)
-- total_cases: float (nullable = true)